# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import time

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../Weatherpy/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp (C),Humidity (%),Cloudiness (%),Wind Speed (m/s),Country,Date
0,0,bredasdorp,-34.5322,20.0403,17.39,71,5,2.24,ZA,1702324887
1,1,wailua homesteads,22.0669,-159.3780,25.48,85,75,3.09,US,1702324887
2,2,tamanrasset,22.7850,5.5228,16.95,17,0,3.60,DZ,1702324888
3,3,new norfolk,-42.7826,147.0587,10.95,80,92,0.72,AU,1702324888
4,4,severo-yeniseyskiy,60.3725,93.0408,-40.24,100,84,0.45,RU,1702324888


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
cities_map = city_data_df.hvplot.points(
"Lng",
"Lat",
geo = True,
size = "Humidity (%)",
scale = .8,
color = "City",
tiles = "OSM",
frame_width = 800,
frame_height = 600,
alpha=0.5)

# Display the map
cities_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity (%))

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

ideal_city_df = city_data_df.loc[(city_data_df['Max Temp (C)']<=27)    #Max temp less than or equal to 27
                            &(city_data_df['Max Temp (C)']>=14)        #Max temp greater than or equal to 14
                            &(city_data_df['Cloudiness (%)']<=50)      #Cloudiness less than or equal to 50
                            &(city_data_df['Humidity (%)']<55)         #Humidity less than 55
                            &(city_data_df['Humidity (%)']>25)         #Humidity more than 25
                            &(city_data_df['Wind Speed (m/s)']<3.5),:]   #Wind speed less than 3.5 m/s


# Drop any rows with null values
ideal_city_df.dropna()
city_count = len(ideal_city_df)

# Display sample data
print (f'Total ideal cities: {city_count}')
ideal_city_df.head()

Total ideal cities: 16


,City_ID,City,Lat,Lng,Max Temp (C),Humidity (%),Cloudiness (%),Wind Speed (m/s),Country,Date
107,107,ibshaway,29.3580,30.6814,17.00,43,0,3.14,EG,1702324920
127,127,konarka,19.9000,86.1167,16.22,53,0,2.69,IN,1702324924
128,128,salalah,17.0151,54.0924,23.32,32,21,1.59,OM,1702324925
131,131,khairabad,27.5333,80.7500,14.12,33,0,1.65,IN,1702324925
135,135,galliano,29.4422,-90.2992,16.11,31,0,2.06,US,1702324926


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df.iloc[:,[1,8,2,3,5]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity (%),Hotel Name
107,ibshaway,EG,29.3580,30.6814,43,
127,konarka,IN,19.9000,86.1167,53,
128,salalah,OM,17.0151,54.0924,32,
131,khairabad,IN,27.5333,80.7500,33,
135,galliano,US,29.4422,-90.2992,31,
203,springbok,ZA,-29.6643,17.8865,34,
255,essaouira,MA,31.5125,-9.7700,45,
275,efon-alaaye,NG,7.6500,4.9167,50,
290,siwana,IN,25.6333,72.4167,28,
303,metlili chaamba,DZ,32.2667,3.6333,32,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories':'accommodation.hotel',
          'apiKey':geoapify_key,
          'limit':1}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lng = row['Lng']
    lat = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f'proximity:{lng},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ibshaway - nearest hotel: No hotel found
konarka - nearest hotel: Hotel Sun Village
salalah - nearest hotel: Muscat International Hotel
khairabad - nearest hotel: Revoli Garden
galliano - nearest hotel: Fairfield Inn & Suites by Marriott Cut Off-Galliano
springbok - nearest hotel: Elkoweru Guesthouse
essaouira - nearest hotel: Essaouira Wind Palace
efon-alaaye - nearest hotel: No hotel found
siwana - nearest hotel: No hotel found
metlili chaamba - nearest hotel: No hotel found
raxaul - nearest hotel: Hotel New Kaveri
geraldton - nearest hotel: Ocean Centre Hotel
al kharijah - nearest hotel: Kharga Hotel
amaigbo - nearest hotel: No hotel found
karratha - nearest hotel: Karratha International Hotel
harar - nearest hotel: Ejersa Hotel


,City,Country,Lat,Lng,Humidity (%),Hotel Name
107,ibshaway,EG,29.3580,30.6814,43,No hotel found
127,konarka,IN,19.9000,86.1167,53,Hotel Sun Village
128,salalah,OM,17.0151,54.0924,32,Muscat International Hotel
131,khairabad,IN,27.5333,80.7500,33,Revoli Garden
135,galliano,US,29.4422,-90.2992,31,Fairfield Inn & Suites by Marriott Cut Off-Gal...
203,springbok,ZA,-29.6643,17.8865,34,Elkoweru Guesthouse
255,essaouira,MA,31.5125,-9.7700,45,Essaouira Wind Palace
275,efon-alaaye,NG,7.6500,4.9167,50,No hotel found
290,siwana,IN,25.6333,72.4167,28,No hotel found
303,metlili chaamba,DZ,32.2667,3.6333,32,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
#%%capture --no-display

# Configure the map plot
hotels_map = hotel_df.hvplot.points(
"Lng",
"Lat",
geo = True,
size = "Humidity (%)",
scale = 1.5,
color = "City",
hover_cols = ["Hotel Name", "Country"],
frame_width = 800,
frame_height = 600,
tiles = "OSM",
alpha=0.5)

# Display the map
hotels_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity (%),Hotel Name,Country)